In [1]:
%matplotlib inline
import os
from time import sleep

import matplotlib.pyplot as plt
import numpy as np
import qcodes as qc
import time
from qcodes import (
    Measurement,
    experiments,
    initialise_database,
    initialise_or_create_database_at,
    load_by_guid,
    load_by_run_spec,
    load_experiment,
    load_last_experiment,
    load_or_create_experiment,
    new_experiment,
)
from qcodes.dataset.plotting import plot_dataset
from qcodes.logger.logger import start_all_logging
from qcodes.tests.instrument_mocks import DummyInstrument

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\ed261978\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\ed261978\.qcodes\logs\200630-15832-qcodes.log


In [2]:
station = qc.Station()

In [3]:
# A dummy instrument dac with two parameters ch1 and Ch2
dac = DummyInstrument('dac', gates=['ch1', 'ch2'])

# A dummy instrument dmm with two parameters ch1 and Ch2
dmm = DummyInstrument('dmm', gates=['v1', 'v2'])

In [4]:
class dmm_parameter(qc.Parameter):
    def __init__(self, name):
        super().__init__(name)
        self.ed = self.exponential_decay(5, 0.2)
        next(self.ed)

    def get_raw(self):
        """
        This method is automatically wrapped to
        provide a ``get`` method on the parameter instance.
        """
        val = self.ed.send(dac.ch1())
        next(self.ed)
        return val

    @staticmethod
    def exponential_decay(a: float, b: float):
        """
        Yields a*exp(-b*x) where x is put in
        """
        x = 0
        while True:
            x = yield
            yield a*np.exp(-b*x) + 0.02*a*np.random.randn()


dmm.v1 = dmm_parameter('dmm_v1')
dmm.v2 = dmm_parameter('dmm_v2')

In [5]:
initialise_or_create_database_at("livePlot.db")

In [6]:
exp = load_or_create_experiment(experiment_name='dataset_context_manager',
                                sample_name="no sample1")

In [7]:
meas = Measurement(exp=exp, station=station)
meas.register_parameter(dac.ch1)  # register the first independent parameter
meas.register_parameter(dmm.v1, setpoints=(dac.ch1,))  # now register the dependent oone
meas.register_parameter(dmm.v2, setpoints=(dac.ch1,))  # now register the dependent oone

meas.write_period = 2

with meas.run() as datasaver:
    for set_v in np.linspace(0, 25, 10):
        dac.ch1.set(set_v)
        get_v = dmm.v1.get()
        datasaver.add_result((dac.ch1, set_v),
                             (dmm.v1, get_v),
                             (dmm.v2, get_v*2.))
        time.sleep(1)
        print(set_v)

    dataset = datasaver.dataset  # convenient to have for plotting

Starting experimental run with id: 40. 
0.0
2.7777777777777777
5.555555555555555
8.333333333333332
11.11111111111111
13.88888888888889
16.666666666666664
19.444444444444443
22.22222222222222
25.0


## 2D

In [16]:
meas = Measurement(exp=exp, station=station)
meas.register_parameter(dac.ch1)
meas.register_parameter(dac.ch2)
meas.register_parameter(dmm.v1, setpoints=(dac.ch1, dac.ch2))  # now register the dependent oone
meas.register_parameter(dmm.v2, setpoints=(dac.ch1, dac.ch2))  # now register the dependent oone

meas.write_period = 2

with meas.run() as datasaver:
    for set_v1 in np.linspace(0, 25, 5):
        dac.ch1.set(set_v1)
        for set_v2 in np.linspace(0, 100, 20):
            dac.ch2.set(set_v2)
            get_v = dmm.v1.get()
            datasaver.add_result((dac.ch1, set_v1),
                                 (dac.ch2, set_v2),
                                 (dmm.v1, get_v),
                                 (dmm.v2, get_v*2))
        
            time.sleep(1)

    dataset = datasaver.dataset  # convenient to have for plotting

Starting experimental run with id: 66. 
